# Bookish: Paso 3
Esta es la etapa final, donde en base a los datasets que tenemos creamos un sistema de recomendacion

In [26]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [27]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [28]:
top1000 = drive.CreateFile({'id':'1IM-NfHu8H8ZDbjmAQPx-L3rWUYJMmySz'})
books_clean = drive.CreateFile({'id':'18wJwL7EqyYQ-sWe0LKGZKVUyCIzUC_xm'})
ratingDownloaded = drive.CreateFile({'id':'1UN6dXHIjsiPjAgmiEdOKoSMJb1fQxMu5'})

In [29]:
top1000.GetContentFile('top1000.csv')
books_clean.GetContentFile('books_clean.csv')
ratingDownloaded.GetContentFile('ratings.csv')

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output

import sklearn.metrics as metrics
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation
from sklearn.metrics.pairwise import pairwise_distances
from contextlib import contextmanager

import warnings
warnings.filterwarnings('ignore')
import os, sys
import re

pd.set_option('display.max_colwidth', -1)

In [31]:
books = pd.read_csv('top1000.csv')
books.head()

,User-ID,000649840X,0020442203,002542730X,0028604199,006000438X,0060008032,0060096195,006016848X,0060173289,0060175400,0060175966,0060188731,006019491X,0060199652,0060248025,0060391626,0060392452,0060502258,0060512822,0060915544,0060916508,0060920084,0060921145,0060922532,0060926317,0060928336,0060929790,0060929871,006092988X,0060930187,0060930535,0060931418,0060932759,0060934417,0060938455,0060958022,0060959037,0060964049,0060976845,...,0877017883,0887307876,0894803204,0894805770,0894808249,089480829X,0971880107,140003065X,1400031346,1400031354,1400031362,1400032717,1400034779,1551668998,1558531025,155874262X,1558743669,1558744150,1558744630,1558745157,1558745718,1559029838,1565122968,1573221937,1573225517,1573225789,1573227331,1573228214,1573229326,1573229571,1576737330,1592400876,1844262553,1857022424,1878424319,1931561648,3257229534,3404148665,3423202327,3492045170
0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
books_all = pd.read_csv('books_clean.csv')
books_all.head()

,Unnamed: 0,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,3,0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,Gina Bari Kolata,1999,Farrar Straus Giroux
4,4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [33]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,Unnamed: 0,User-ID,ISBN,Book-Rating
0,0,276725,034545104X,0
1,1,276726,0155061224,5
2,2,276727,0446520802,0
3,3,276729,052165615X,3
4,4,276729,0521795028,6


# Entrenando nuestro sistema de recomendacion(Primero)

In [34]:
#variables globales
global metric,k
k=10
metric='cosine'

# Item Based

In [35]:
#Esta funcion encuentra items(libros) libros similares con el item_id y la matriz de calificaciones

def findksimilaritems(item_id, ratings, metric=metric, k=k):
    similarities=[]
    indices=[]
    ratings=ratings.T
    loc = ratings.index.get_loc(item_id)
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute')
    model_knn.fit(ratings)
    
    distances, indices = model_knn.kneighbors(ratings.iloc[loc, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()

    return similarities,indices

In [36]:
#Esta funcion predice el rating 
def predict_itembased(user_id, item_id, ratings, metric = metric, k=k):
    prediction= wtd_sum =0
    user_loc = ratings.index.get_loc(user_id)
    item_loc = ratings.columns.get_loc(item_id)
    similarities, indices=findksimilaritems(item_id, ratings) #similar users based on correlation coefficients
    sum_wt = np.sum(similarities)-1
    product=1
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] == item_loc:
            continue;
        else:
            product = ratings.iloc[user_loc,indices.flatten()[i]] * (similarities[i])
            wtd_sum = wtd_sum + product                              
    prediction = int(round(wtd_sum/sum_wt))
    
    #en caso de tener un dataset muy escaso, usar la metrica de correlacion para 'collaborative based approach' puede dar calificaciones negativas
    #aqui manejamos ese caso
    #predicciones en caso de un dataset muy escaso
    if prediction <= 0:
        prediction = 1   
    elif prediction >10:
        prediction = 10

    print('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))      
    
    return prediction

In [37]:
prediction = predict_itembased(42,'0020442203',books)


Predicted rating for user 42 -> item 0020442203: 1


In [38]:
from contextlib import contextmanager

In [39]:
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [40]:
#Esta función utiliza las funciones anteriores para recomendar elementos para el enfoque basado en el item/usuario y el coseno/correlación.
#Se hacen recomendaciones si la calificación prevista para un elemento es> = a 6 y los elementos aún no se han calificado
def recommendItem(user_id, ratings, metric=metric):    
    if (user_id not in ratings.index.values) or type(user_id) is not int:
        print("User id should be a valid integer from this list :\n\n {} ".format(re.sub('[\[\]]', '', np.array_str(books.index.values))))
    else:    
        ids = ['Item-based (correlation)','Item-based (cosine)']
        select = widgets.Dropdown(options=ids, value=ids[0],description='Select approach', width='1000px')
        def on_change(change):
            clear_output(wait=True)
            prediction = []            
            if change['type'] == 'change' and change['name'] == 'value':            
                if (select.value == 'Item-based (correlation)') :
                    metric = 'correlation'
                else:                       
                    metric = 'cosine'   
                with suppress_stdout():
                    if (select.value == 'Item-based (correlation)') | (select.value == 'Item-based (cosine)'):
                        for i in range(ratings.shape[1]):
                            if (ratings[str(ratings.columns[i])][user_id] !=0): #not rated already
                                prediction.append(predict_itembased(user_id, str(ratings.columns[i]) ,ratings, metric))
                            else:                    
                                prediction.append(-1) #for already rated items
                    else:
                        for i in range(ratings.shape[1]):
                            if (ratings[str(ratings.columns[i])][user_id] !=0): #not rated already
                                prediction.append(predict_userbased(user_id, str(ratings.columns[i]) ,ratings, metric))
                            else:                    
                                prediction.append(-1) #for already rated items
                prediction = pd.Series(prediction)
                prediction = prediction.sort_values(ascending=False)
                recommended = prediction[:10]
                print("As per {0} approach....Following books are recommended...".format(select.value))
                for i in range(len(recommended)):
                     print("{0}. {1}".format(i+1,books_all.bookTitle[recommended.index[i]].encode('utf-8')))                        
        select.observe(on_change)
        display(select)

In [41]:
#checando si funciona cuando dan entradas incorrectas
recommendItem(999999,books)

User id should be a valid integer from this list :

     0     1     2 ... 25450 25451 25452 


In [42]:
recommendItem(4385, books)

As per Item-based (cosine) approach....Following books are recommended...
1. b'Classical Mythology'
2. b'James and the Giant Peach'
3. b'The Music Box'
4. b'The Best Democracy Money Can Buy: The Truth About Corporate Cons, Globalization and High-Finance Fraudsters'
5. b'Desert Notes/River Notes'
6. b'Diario de Ana Frank'
7. b'Affinity'
8. b"Vinegar Hill (Oprah's Book Club (Paperback))"
9. b'Last Chance to See'
10. b'ALL THAT REMAINS'


# Entrenando nuestro sistema de recomendacion(Segundo)

In [ ]:
booksAll = books_all
booksAll.head()

,Unnamed: 0,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,3,0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,Gina Bari Kolata,1999,Farrar Straus Giroux
4,4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [ ]:
#El conjunto de datos de calificaciones debe tener solo los libros que existen en nuestro conjunto de datos de libros
ratings_new = ratings[ratings.ISBN.isin(booksAll.ISBN)]

ratings.shape,ratings_new.shape

((1149780, 4), (1031136, 4))

In [ ]:
#Separamos las calificaciones implicitas y explicitas
ratings_explicit = ratings_new[ratings_new['Book-Rating'] != 0]

In [ ]:
print(ratings_new.shape)
print(ratings_explicit.shape)

(1031136, 4)
(383842, 4)


In [ ]:
ratings_count_prom = pd.DataFrame(ratings_explicit.groupby(['ISBN'])['Book-Rating'].mean())
print(ratings_count_prom)

ratings_count = pd.DataFrame(ratings_explicit.groupby(['ISBN'])['Book-Rating'].sum())
booksRating = ratings_count.sort_values('Book-Rating', ascending = False).head(1000)

            Book-Rating
ISBN                   
0000913154  8.0        
0001046438  9.0        
000104687X  6.0        
0001047213  9.0        
0001047973  9.0        
...         ...        
B0001FZGPI  7.0        
B0001FZGRQ  9.0        
B0001GMSV2  8.0        
B0001I1KOG  10.0       
B000234N3A  9.0        

[149836 rows x 1 columns]


In [ ]:
booksRatingAvg = booksRating.merge(ratings_count_prom, left_index = True, right_on = 'ISBN').merge(booksAll, left_index = True, right_on = 'ISBN')
booksRatingAvg

,Book-Rating_x,Book-Rating_y,Unnamed: 0,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
408,5787,8.185290,408,0316666343,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown"
748,4108,8.435318,748,0385504209,The Da Vinci Code,Dan Brown,2003,Doubleday
522,3134,8.182768,522,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA
2143,2798,8.939297,2143,059035342X,Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),J. K. Rowling,1999,Arthur A. Levine Books
356,2595,8.452769,356,0142001740,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books
...,...,...,...,...,...,...,...,...
5255,250,8.333333,5255,0679732411,Gift from the Sea,Anne Morrow Lindbergh,1991,Pantheon Books
21201,250,8.064516,21201,0312978383,Winter Solstice,Rosamunde Pilcher,2001,St. Martin's Paperbacks
1431,250,8.333333,1431,0345325818,The Silmarillion,J.R.R. TOLKIEN,1985,Del Rey
15967,250,7.812500,15967,0345447867,The Apprentice,Tess Gerritsen,2003,Ballantine Books


In [ ]:
import math 

In [ ]:
booksRatingAvg.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 408 to 39129
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Book-Rating_x      1000 non-null   int64  
 1   Book-Rating_y      1000 non-null   float64
 2   Unnamed: 0         1000 non-null   int64  
 3   ISBN               1000 non-null   object 
 4   bookTitle          1000 non-null   object 
 5   bookAuthor         1000 non-null   object 
 6   yearOfPublication  1000 non-null   int64  
 7   publisher          1000 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 70.3+ KB


In [ ]:
#aqui vamos a poder comparar la similitud entre el libro escogido y el resto de los libros del dataset
def Similarity(bookId1, bookId2):
    a = booksRatingAvg.loc[(booksRatingAvg.ISBN == bookId1),:]
    b = booksRatingAvg.loc[(booksRatingAvg.ISBN == bookId2),:]
    
    yearA = a['yearOfPublication']
    yearB = b['yearOfPublication']
    
    yearDistance = int(math.pow((int(math.pow (yearA, 2)) - int(math.pow (yearB, 2))),2))
    
    authorA = a['bookAuthor']
    authorB = b['bookAuthor']
    if authorA is authorB:
      authorDistance = 0
    else:
      authorDistance = 1
    #print(authorDistance)

    return math.sqrt(yearDistance+authorDistance)

In [ ]:
Similarity('0316666343','0385504209')

4005.000124843943

In [ ]:
booksAll.loc[(booksAll.ISBN == '0195153448'),:]

,Unnamed: 0,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press


In [ ]:
booksRatingAvg.loc[(booksRatingAvg.ISBN == '0385504209'),:]

,Book-Rating_x,Book-Rating_y,Unnamed: 0,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
748,4108,8.435318,748,0385504209,The Da Vinci Code,Dan Brown,2003,Doubleday


In [ ]:
#aqui vamos a llamar nuestra funcion de similitud, y traer los 10 libros mas similares junto con su rating  
import operator
def predict_score():
    name = input('Enter a book title: ')
    new_book = booksRatingAvg[booksRatingAvg['ISBN'].str.contains(name)].iloc[0].to_frame().T
    print('Selected Book: ',new_book.ISBN.values[0])
    def getNeighbors(baseBook, K):
        distances = []
        
        for index, book in booksRatingAvg.iterrows():
            if index != baseBook.index.values[0]:
                dist = Similarity(baseBook['ISBN'].values[0], book['ISBN'])
                distances.append((index, dist))
    
        distances.sort(key=operator.itemgetter(1))
        neighbors = []
    
        for x in range(K):
            neighbors.append(distances[x])
        return neighbors
    
    K = 10
    avgRating = 0
    neighbors = getNeighbors(new_book, K)
    print('\nRecommended Books: \n')
    for neighbor in neighbors:
        avgRating = avgRating+booksRatingAvg.loc[neighbor[0]]['Book-Rating_y']  
        print( str(booksRatingAvg.loc[neighbor[0]]['bookTitle']) +" | Rating: "+str(booksRatingAvg.loc[neighbor[0]]['Book-Rating_y']))
    

In [ ]:
predict_score()

Enter a book title: 0385504209
Selected Book:  0385504209

Recommended Books: 

The Secret Life of Bees | Rating: 8.452768729641694
The Five People You Meet in Heaven | Rating: 8.016528925619834
Harry Potter and the Order of the Phoenix (Book 5) | Rating: 9.033980582524272
Life of Pi | Rating: 8.030973451327434
The No. 1 Ladies' Detective Agency (Today Show Book Club #8) | Rating: 8.049180327868852
The Nanny Diaries: A Novel | Rating: 7.293413173652695
Dance upon the Air (Three Sisters Island Trilogy) | Rating: 8.038095238095238
The Beach House | Rating: 7.836538461538462
Dude, Where's My Country? | Rating: 8.337078651685394
The King of Torts | Rating: 7.619565217391305


In [ ]:
booksRatingAvg.to_csv('booksRatingAvg.csv')